# Explore here

It's recommended to use this notebook for exploration purposes.

For example: 

1. You could import the CSV generated by python into your notebook and explore it.
2. You could connect to your database using `pandas.read_sql` from this notebook and explore it.

In [1]:
!pip install -r ../requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 19.0 MB/s eta 0:00:00m eta 0:00:010:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.7/11.7 MB 55.1 MB/s eta 0:00:00m eta 0:00:010:01:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 32.7 MB/s eta 0:00:00m eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 57.9 MB/s eta 0:00:00m eta 0:00:010:0101
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.9/60.9 MB 20.8 MB/s eta 0:00:00m eta 0:00:010:01:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 292.8/292.8 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.9/192.9 MB 7.3 MB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 60.6 MB/s eta 0:00:00m eta 0:00:010:01:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 52.0 MB/s 

In [42]:
import matplotlib.pyplot as plt

import numpy as np 
import pandas as pd
import seaborn as sns
import pickle
import xgboost as xgb

from xgboost import XGBClassifier
from sklearn.ensemble import  RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from urllib.request import urlretrieve
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV



In [3]:
df_raw = pd.read_csv('https://raw.githubusercontent.com/4GeeksAcademy/random-forest-project-tutorial/main/titanic_train.csv') 
print(df_raw)

     PassengerId  Survived  Pclass  \
0              1         0       3   
1              2         1       1   
2              3         1       3   
3              4         1       1   
4              5         0       3   
..           ...       ...     ...   
886          887         0       2   
887          888         1       1   
888          889         0       3   
889          890         1       1   
890          891         0       3   

                                                  Name     Sex   Age  SibSp  \
0                              Braund, Mr. Owen Harris    male  22.0      1   
1    Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
2                               Heikkinen, Miss. Laina  female  26.0      0   
3         Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0      1   
4                             Allen, Mr. William Henry    male  35.0      0   
..                                                 ...     ...   ... 

In [4]:
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [5]:
df_raw.sample(10)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
174,175,0,1,"Smith, Mr. James Clinch",male,56.0,0,0,17764,30.6958,A7,C
413,414,0,2,"Cunningham, Mr. Alfred Fleming",male,NaN,0,0,239853,0.0000,NaN,S
168,169,0,1,"Baumann, Mr. John D",male,NaN,0,0,PC 17318,25.9250,NaN,S
402,403,0,3,"Jussila, Miss. Mari Aina",female,21.0,1,0,4137,9.8250,NaN,S
308,309,0,2,"Abelson, Mr. Samuel",male,30.0,1,0,P/PP 3381,24.0000,NaN,C
19,20,1,3,"Masselmani, Mrs. Fatima",female,NaN,0,0,2649,7.2250,NaN,C
533,534,1,3,"Peter, Mrs. Catherine (Catherine Rizk)",female,NaN,0,2,2668,22.3583,NaN,C
383,384,1,1,"Holverson, Mrs. Alexander Oskar (Mary Aline To...",female,35.0,1,0,113789,52.0000,NaN,S
450,451,0,2,"West, Mr. Edwy Arthur",male,36.0,1,2,C.A. 34651,27.7500,NaN,S
64,65,0,1,"Stewart, Mr. Albert A",male,NaN,0,0,PC 17605,27.7208,NaN,C


In [6]:
df_raw['Sex'].value_counts()

male      577
female    314
Name: Sex, dtype: int64

In [7]:
df_raw['Embarked'].value_counts()

S    644
C    168
Q     77
Name: Embarked, dtype: int64

In [8]:
df_raw.info

<bound method DataFrame.info of      PassengerId  Survived  Pclass  \
0              1         0       3   
1              2         1       1   
2              3         1       3   
3              4         1       1   
4              5         0       3   
..           ...       ...     ...   
886          887         0       2   
887          888         1       1   
888          889         0       3   
889          890         1       1   
890          891         0       3   

                                                  Name     Sex   Age  SibSp  \
0                              Braund, Mr. Owen Harris    male  22.0      1   
1    Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
2                               Heikkinen, Miss. Laina  female  26.0      0   
3         Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0      1   
4                             Allen, Mr. William Henry    male  35.0      0   
..                                   

Borramos las columnas que no vamos a precisar

In [9]:
df_transform=df_raw.drop(['Cabin','PassengerId','Ticket','Name'],axis=1)

In [10]:
# dos variables nuevas
df_transform['Sex_encoded']=df_transform['Sex'].apply(lambda x: 1 if x=="female" else 0)

In [11]:
df_transform = df_transform.drop(['Sex'],axis=1)

In [12]:
df_transform['Embarked_S']=df_transform['Embarked'].apply(lambda x: 1 if x=="S" else 0)

In [13]:
df_transform['Embarked_C']=df_transform['Embarked'].apply(lambda x: 1 if x=="C" else 0)

In [14]:
df_transform['Age_clean']=df_transform['Age'].fillna(30)

In [15]:
df_transform=df_transform.drop(['Embarked'],axis=1)
df_transform=df_transform.drop(['Age'],axis=1)

In [16]:
df=df_transform.copy()

In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Survived     891 non-null    int64  
 1   Pclass       891 non-null    int64  
 2   SibSp        891 non-null    int64  
 3   Parch        891 non-null    int64  
 4   Fare         891 non-null    float64
 5   Sex_encoded  891 non-null    int64  
 6   Embarked_S   891 non-null    int64  
 7   Embarked_C   891 non-null    int64  
 8   Age_clean    891 non-null    float64
dtypes: float64(2), int64(7)
memory usage: 62.8 KB


In [18]:
X=df.drop(['Survived'],axis=1)

In [19]:
y=df['Survived']

In [20]:
X_train,X_test,y_train,y_test = train_test_split(X,y,random_state=70)

In [21]:
# Fit a Decision Tree model as comparison

clf = DecisionTreeClassifier()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
accuracy_score(y_test, y_pred)

0.7937219730941704

In [22]:
# Fit a Random Forest model

clf = RandomForestClassifier(n_estimators=100, max_features="auto",random_state=0)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
accuracy_score(y_test, y_pred)

/home/gitpod/.pyenv/versions/3.8.13/lib/python3.8/site-packages/sklearn/ensemble/_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(


0.7982062780269058

In [23]:
# Step 6: Fit a Gradient Boosting model

clf = GradientBoostingClassifier(n_estimators=100)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
accuracy_score(y_test, y_pred)



0.8295964125560538

Por ahora el GradientBoosting es el mejor resultado con 0.83 de accurancy_score

In [28]:
D_train = xgb.DMatrix(X_train, label=y_train, enable_categorical=True)
D_test = xgb.DMatrix(X_test, label=y_test, enable_categorical=True)

In [29]:
D_train

Parametros a seleccionar

booster: El tipo de modelo de clasificación usado, por defecto gbtree.
objective: El tipo de tarea de clasificación que realizaremos. Para clasificación binaria, nuestro caso, especificamos binary:logistic.
max_depth: “Profundidad” o número de nodos de bifurcación de los árboles de decisión usados en el entrenamiento. Aunque una mayor profundidad puede devolver mejores resultados, también puede resultar en overfitting (sobre ajuste).
eta: La tasa de aprendizaje del modelo. Un mayor valor llega más rápidamente al mínimo de la función objetivo, es decir, a un “mejor modelo”, pero puede “pasarse” de su valor óptimo. En cambio, un valor pequeño puede nunca llegar al valor óptimo de la función objetivo, incluso después de muchas iteraciones. En ambos casos, esto afecta el desempeño de nuestros modelos con nuevos.
nthread: El número de hilos computacionales que serán usados en el proceso de entrenamiento. Generalmente se refiere a los núcleos del procesador de tu equipo de cómputo, local o remoto, pero también pueden ser los núcleos de un GPU.
nround: El número de iteraciones que se realizarán antes de detener el proceso de ajuste. Un mayor número de iteraciones generalmente devuelve mejores resultados de predicción, pero necesita más tiempo de entrenamiento y conlleva un riesgo de sobre ajuste si son demasiadas rondas.

In [30]:
parametros = {"booster":"gbtree", "max_depth": 2, "eta": 0.3, "objective": "binary:logistic", "nthread":2}
rondas = 10

In [31]:
evaluacion = [(D_test, "eval"), (D_train, "train")]

Con estas variables definidas, continuamos con el entrenamiento del modelo. Notarás que se irán mostrando los resultados de evaluación de cada iteración del modelo, hasta que alcance el número de rondas que hemos definido.

In [32]:
modelo = xgb.train(parametros, D_train, rondas, evaluacion)

/home/gitpod/.pyenv/versions/3.8.13/lib/python3.8/site-packages/xgboost/core.py:525: FutureWarning: Pass `evals` as keyword args.  Passing these as positional arguments will be considered as error in future releases.
  warnings.warn(


[0]	eval-logloss:0.58920	train-logloss:0.58220
[1]	eval-logloss:0.53747	train-logloss:0.52306
[2]	eval-logloss:0.49385	train-logloss:0.48806
[3]	eval-logloss:0.48303	train-logloss:0.46617
[4]	eval-logloss:0.46340	train-logloss:0.45297
[5]	eval-logloss:0.45382	train-logloss:0.44076
[6]	eval-logloss:0.44218	train-logloss:0.43088
[7]	eval-logloss:0.44012	train-logloss:0.42523
[8]	eval-logloss:0.43525	train-logloss:0.41942
[9]	eval-logloss:0.43272	train-logloss:0.41407


Generacion de Predicciones

In [33]:
prediccion = modelo.predict(D_test)

In [34]:
prediccion

array([0.9250106 , 0.95114714, 0.8693489 , 0.17490175, 0.17490175,
       0.18023612, 0.8970491 , 0.8785872 , 0.21051596, 0.17490175,
       0.54657185, 0.17490175, 0.17490175, 0.14877917, 0.12675628,
       0.18130095, 0.3768865 , 0.14877917, 0.14877917, 0.14877917,
       0.9034693 , 0.10812957, 0.30748942, 0.8970491 , 0.8745533 ,
       0.2813915 , 0.14877917, 0.32292524, 0.9034693 , 0.14877917,
       0.95114714, 0.8410089 , 0.18130095, 0.9034693 , 0.8467681 ,
       0.14877917, 0.12675628, 0.14877917, 0.93659914, 0.11522276,
       0.9034693 , 0.51526344, 0.20244259, 0.2460344 , 0.70040876,
       0.12675628, 0.30748942, 0.11522276, 0.32453743, 0.63643867,
       0.14877917, 0.8970491 , 0.570457  , 0.20244259, 0.30748942,
       0.54657185, 0.9250106 , 0.47641724, 0.6025949 , 0.14877917,
       0.23125269, 0.14877917, 0.1929879 , 0.58501273, 0.26940903,
       0.54657185, 0.9250106 , 0.12675628, 0.14877917, 0.14877917,
       0.9034693 , 0.14877917, 0.18130095, 0.95114714, 0.21051

In [35]:
prediccion = [1 if i > .7 else 0 for i in prediccion]
prediccion[:10]

[1, 1, 1, 0, 0, 0, 1, 1, 0, 0]

Ahora vamos a ver que tan acertadas estan nuestras predicciones

In [37]:
def metricas(objetivo, prediccion):
    matriz_conf = confusion_matrix(objetivo, prediccion)
    score = accuracy_score(objetivo, prediccion)
    reporte = classification_report(objetivo, prediccion)
    metricas = [matriz_conf, score, reporte]
    return(metricas)

In [43]:
metricas = metricas(y, prediccion)
[print(i) for i in metricas]

ValueError: Found input variables with inconsistent numbers of samples: [891, 223]

In [24]:



parametros = {
    'eta': 0.2,     # 0.3 por defecto
    'max_depth': 4,  # 6 por defecto
    'objective': 'multi:softmax',  
    'num_class': 2} 

steps = 20

XGB = xgb.train(parametros, D_train, steps)

In [26]:
#XGB.fit(X_train, y_train)
y_pred = XGB.predict(X_test)
accuracy_score(y_test, y_pred)

TypeError: ('Expecting data to be a DMatrix object, got: ', <class 'pandas.core.frame.DataFrame'>)

In [24]:


D_train = xgb.DMatrix(X_train, label=Y_train)
D_test = xgb.DMatrix(X_test, label=Y_test)

param = {
    'eta': 0.3, 
    'max_depth': 3,  
    'objective': 'multi:softprob',  
    'num_class': 3} 

steps = 10  # The number of training iterations

model = xgb.train(param, D_train, steps)

In [25]:

clf = xgb.XGBClassifier()
parameters = {
     "eta"    : [0.05, 0.10, 0.15, 0.20, 0.25, 0.30 ] ,
     "max_depth"        : [ 3, 4, 5, 6, 8, 10, 12, 15],
     "min_child_weight" : [ 1, 3, 5, 7 ],
     "gamma"            : [ 0.0, 0.1, 0.2 , 0.3, 0.4 ],
     "colsample_bytree" : [ 0.3, 0.4, 0.5 , 0.7 ]
     }

grid = GridSearchCV(clf,
                    parameters, n_jobs=4,
                    scoring="neg_log_loss",
                    cv=3)

grid.fit(X_train, Y_train)